# b) Ngonye Falls Synthetic Historic Flow Series

To produce a long time series (1924-2017) of daily flow for the Ngonye Falls site, a correlation is carried out between the overlapping portions of the *Zambezi River Authority* (ZRA) gauge at Ngonye Falls and portion of the record of the Victoria Falls gauge (2005-2016).

Once correlated, the full Victoria Falls gauge record is extrapolated to produce a synthetic record for Ngonye Falls.

## Procedure

From: **Mott MacDonald** - *Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D*

1. Calculate Flow Duration Curves (FDCs) from daily data for the following:
    1. The target site (Ngonye) for 2005/06 – 2016/17; 
    2. The analogue site (Victoria Falls) for the exact period of data at the target site (2005/06 – 2016/17);
    3. Analogue site for long-term record (1924/25 -2016/17); 
    
    
2. Compare the FDCs for the analogue site for the two periods and derive factors between the shorter and longer periods; 

3. Apply these factors to the target site FDC to produce an estimated long-term FDC for the target site;

4. For each daily flow value at the analogue site, determine its position on the long-term FDC (percentile); and,

5. Look up the flow value for this percentile from the estimated long-term FDC for the target site taking into account of the 11-day time lag.

*Water Year* is defined as running Oct to Sept as is usual in the region.

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| daily_gauge_vicfalls.csv        | Zambezi River Authority | Daily Flow for Victoria Falls 1924/25 - 2016/17              |
| daily_gauge_ngonye.csv | Zambezi River Authority   | Daily Stage (Level) for Ngonye Falls 2005/06 – 2016/17 |
|  Stage - Discharge equation for Ngonye Falls |  Mott MacDonald |Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D |


## Outputs
| File                       | Description                                 |
|----------------------------|---------------------------------------------|
| ngonye_synthetic.csv  | Synthetic daily flow series for Ngonye  Falls 1924/25 - 2016/17  |
| ngonye_vicfalls_fdcs.csv           | Flow Duration Curves for Victoria Falls and Ngonye Falls |


## Libraries

In [1]:
import numpy as np
import pandas as pd
import datetime

## Parameters

In [2]:
input_data='./input_data/2024/'
output_data='./output_data/2024/'
#overlap_cutoff=datetime.datetime(2017, 9,30)
#overlap_cutoff=datetime.datetime(2022, 10,11)
overlap_cutoff=datetime.datetime(2024, 9,30)

## Load Data
Load the Vic Falls data and add some helper columns.

In [4]:
vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls_2024.csv")
#vicfalls=pd.read_csv(input_data + "daily_gauge_vicfalls.csv")

vicfalls['Date']=pd.to_datetime(vicfalls['Date'],format="%d/%m/%Y")
vicfalls=vicfalls.set_index(pd.DatetimeIndex(vicfalls['Date']))
vicfalls=vicfalls.drop(['Date'],axis=1)

vicfalls=vicfalls.astype({'Flow': 'float64'})
vicfalls['Flow']=np.round(vicfalls['Flow'],4)

vicfalls

,Flow
Date,
1924-10-01,100.0
1924-10-02,100.0
1924-10-03,100.0
1924-10-04,100.0
1924-10-05,100.0
...,...
2024-09-26,238.4
2024-09-27,238.4
2024-09-28,235.4


Load the Ngonye Falls level data and calculate flow based on the stage-discharge relationship:

\begin{equation*}
flow=1093.0355*(level-2.85)^{1.659}
\end{equation*}

Add some helper columns.

In [6]:
ngo=pd.read_csv(input_data+ "daily_gauge_ngonye_2024.csv")
#ngo=pd.read_csv(input_data+ "daily_gauge_ngonye.csv")
ngo['Date']=pd.to_datetime(ngo['Date'],format="%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo=ngo.drop(['Date'],axis=1)
ngo['Flow']=np.round(1093.0355*(ngo['Level']-2.85)**1.659,4)
ngo['VicFalls']=vicfalls['Flow']
ngo

,Level,Flow,VicFalls
Date,,,
2005-10-01,3.218,208.1497,211.8
2005-10-02,3.216,206.2763,210.5
2005-10-03,3.215,205.3422,208.6
2005-10-04,3.213,203.4789,204.1
2005-10-05,3.212,202.5498,204.1
...,...,...,...
2024-09-26,3.222,211.9166,238.4
2024-09-27,3.218,208.1497,238.4
2024-09-28,3.214,204.4097,235.4


## Flow Duration Curve

Build the Flow Duration Curve table in 0.1% exceedance increments.

Add FDC flows for the full Vic Falls timeseries.

In [7]:
fdc=pd.DataFrame({'Exceedance': np.arange(0,1.001,0.001)}).set_index('Exceedance')
fdc['VicFalls_full']=np.round(np.quantile(vicfalls.loc[vicfalls.index<=overlap_cutoff]['Flow'],1-fdc.index),4)
fdc

,VicFalls_full
Exceedance,
0.000,9435.6000
0.001,8378.3000
0.002,7352.5016
0.003,6630.5172
0.004,6171.6000
...,...
0.996,126.0000
0.997,115.3000
0.998,107.1000


Add flows for Ngonye to the FDC.

Add flows for the portion of the Vic Falls series that overlaps with the Ngonye Series.

In [8]:
fdc['Ngonye_gauged']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['Flow'],((1-fdc.index)*100)),4)
fdc['VicFalls_overlap']=np.round(np.percentile(ngo.loc[ngo.index<=overlap_cutoff]['VicFalls'],((1-fdc.index)*100)),4)
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap
Exceedance,,,
0.000,9435.6000,5935.9208,5650.5000
0.001,8378.3000,5751.4615,5532.4400
0.002,7352.5016,5545.1060,5315.4790
0.003,6630.5172,5340.5073,5162.1953
0.004,6171.6000,5073.0458,4936.6384
...,...,...,...
0.996,126.0000,155.7670,145.9900
0.997,115.3000,154.0979,139.1000
0.998,107.1000,152.4361,124.3194


## Vic Falls Ratio
Calculate the ratio of Vic Falls flows for the whole series and the overallping protion across the FDC.

Smooth that ratio (0.7% moving average) except at the tails of the FDC.

In [9]:
fdc['VicFalls_factor']=fdc['VicFalls_full']/fdc['VicFalls_overlap']
fdc['VicFalls_factor_smooth']=fdc['VicFalls_factor'].rolling(7,center=True).mean()
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name<0.01 else x['VicFalls_factor_smooth'])),axis=1)
fdc['VicFalls_factor_smooth']=fdc.apply((lambda x: (x['VicFalls_factor'] if x.name>0.99 else x['VicFalls_factor_smooth'])),axis=1)
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth
Exceedance,,,,,
0.000,9435.6000,5935.9208,5650.5000,1.669870,1.669870
0.001,8378.3000,5751.4615,5532.4400,1.514395,1.514395
0.002,7352.5016,5545.1060,5315.4790,1.383225,1.383225
0.003,6630.5172,5340.5073,5162.1953,1.284437,1.284437
0.004,6171.6000,5073.0458,4936.6384,1.250162,1.250162
...,...,...,...,...,...
0.996,126.0000,155.7670,145.9900,0.863073,0.863073
0.997,115.3000,154.0979,139.1000,0.828900,0.828900
0.998,107.1000,152.4361,124.3194,0.861491,0.861491


## Scaled FDC
Produce a scaled FDC for Ngonye by using the factors calculated for the Vic Falls data.

In [10]:
fdc['Ngonye_scaled']=fdc['Ngonye_gauged']*fdc['VicFalls_factor_smooth']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled
Exceedance,,,,,,
0.000,9435.6000,5935.9208,5650.5000,1.669870,1.669870,9912.215609
0.001,8378.3000,5751.4615,5532.4400,1.514395,1.514395,8709.985085
0.002,7352.5016,5545.1060,5315.4790,1.383225,1.383225,7670.127328
0.003,6630.5172,5340.5073,5162.1953,1.284437,1.284437,6859.547819
0.004,6171.6000,5073.0458,4936.6384,1.250162,1.250162,6342.131411
...,...,...,...,...,...,...
0.996,126.0000,155.7670,145.9900,0.863073,0.863073,134.438263
0.997,115.3000,154.0979,139.1000,0.828900,0.828900,127.731760
0.998,107.1000,152.4361,124.3194,0.861491,0.861491,131.322274


## Conversion
Calculate a conversion factor for each row of the FDC between the scaled Ngonye FDC flows and the full Vic Falls series FDC.

In [11]:
fdc['Conversion']=fdc['Ngonye_scaled']/fdc['VicFalls_full']
fdc

,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion
Exceedance,,,,,,,
0.000,9435.6000,5935.9208,5650.5000,1.669870,1.669870,9912.215609,1.050512
0.001,8378.3000,5751.4615,5532.4400,1.514395,1.514395,8709.985085,1.039589
0.002,7352.5016,5545.1060,5315.4790,1.383225,1.383225,7670.127328,1.043200
0.003,6630.5172,5340.5073,5162.1953,1.284437,1.284437,6859.547819,1.034542
0.004,6171.6000,5073.0458,4936.6384,1.250162,1.250162,6342.131411,1.027632
...,...,...,...,...,...,...,...
0.996,126.0000,155.7670,145.9900,0.863073,0.863073,134.438263,1.066970
0.997,115.3000,154.0979,139.1000,0.828900,0.828900,127.731760,1.107821
0.998,107.1000,152.4361,124.3194,0.861491,0.861491,131.322274,1.226165


Lookup the conversion factors from the FDC based on flow and give each record in the full Vic Falls series its corresponding conversion factor. 

In [12]:
if 'Conversion' in vicfalls.columns:
    display(vicfalls.columns)
    vicfalls=vicfalls.drop(['Conversion','Exceedance'],axis=1)

#vicfalls['tmp_Flow']=100000-vicfalls['Flow']
#fdc['tmp_Flow']=100000-fdc['VicFalls_full']
fdc['ExceedanceInv']=1-fdc.index

tmp=pd.merge_asof(vicfalls.reset_index().sort_values('Flow',ascending=True),fdc.reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=True),left_on='Flow',right_on='VicFalls_full').set_index('Date')
vicfalls['Conversion']=tmp['Conversion']
vicfalls['Exceedance']=tmp['Exceedance']

vicfalls

,Flow,Conversion,Exceedance
Date,,,
1924-10-01,100.0,1.364952,0.999
1924-10-02,100.0,1.364952,0.999
1924-10-03,100.0,1.364952,0.999
1924-10-04,100.0,1.364952,0.999
1924-10-05,100.0,1.364952,0.999
...,...,...,...
2024-09-26,238.4,0.961365,0.910
2024-09-27,238.4,0.961365,0.910
2024-09-28,235.4,0.963129,0.913


## Ngonye Sythetic
Prepare the full  synthetic series for Ngonye by applying the 11 day lag to the Vic Falls series and the conversion factors calculated previoulsy.

In [25]:

ngonye_synth=pd.DataFrame(index=vicfalls.index)

ngonye_synth['LaggedDate']=ngonye_synth.index+pd.DateOffset(days=11)
ngonye_synth['VicFalls']=ngonye_synth.join(vicfalls,on='LaggedDate')['Flow']
ngonye_synth['Conversion']=ngonye_synth.join(vicfalls,on='LaggedDate')['Conversion']
ngonye_synth['Flow']=ngonye_synth['Conversion'] * ngonye_synth['VicFalls']

ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.364952,136.495189
1924-10-02,1924-10-13,100.0,1.364952,136.495189
1924-10-03,1924-10-14,100.0,1.364952,136.495189
1924-10-04,1924-10-15,100.0,1.364952,136.495189
1924-10-05,1924-10-16,100.0,1.364952,136.495189
...,...,...,...,...
2024-09-26,2024-10-07,NaN,NaN,NaN
2024-09-27,2024-10-08,NaN,NaN,NaN
2024-09-28,2024-10-09,NaN,NaN,NaN


Remove extra columns and delete from the bottom to align to the water year.

In [26]:
ngonye_synth=ngonye_synth.dropna()
ngonye_synth=ngonye_synth.drop(ngonye_synth.loc['2024-10-01':].index)
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow
Date,,,,
1924-10-01,1924-10-12,100.0,1.364952,136.495189
1924-10-02,1924-10-13,100.0,1.364952,136.495189
1924-10-03,1924-10-14,100.0,1.364952,136.495189
1924-10-04,1924-10-15,100.0,1.364952,136.495189
1924-10-05,1924-10-16,100.0,1.364952,136.495189
...,...,...,...,...
2024-09-15,2024-09-26,238.4,0.961365,229.189501
2024-09-16,2024-09-27,238.4,0.961365,229.189501
2024-09-17,2024-09-28,235.4,0.963129,226.720641


Add the new synthetic Ngonye flow series to the FDC.

In [27]:
fdc['Ngonye_synthetic']=np.percentile(ngonye_synth['Flow'],((1-fdc.index)*100))

tmp=pd.merge_asof(ngonye_synth.sort_values('Flow').reset_index(),fdc.sort_values('Ngonye_synthetic').reset_index(),left_on='Flow',right_on='Ngonye_synthetic').set_index('Date')
ngonye_synth['Exceedance']=tmp['Exceedance']

In [28]:
ngonye_synth

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1924-10-01,1924-10-12,100.0,1.364952,136.495189,0.998
1924-10-02,1924-10-13,100.0,1.364952,136.495189,0.998
1924-10-03,1924-10-14,100.0,1.364952,136.495189,0.998
1924-10-04,1924-10-15,100.0,1.364952,136.495189,0.998
1924-10-05,1924-10-16,100.0,1.364952,136.495189,0.998
...,...,...,...,...,...
2024-09-15,2024-09-26,238.4,0.961365,229.189501,0.910
2024-09-16,2024-09-27,238.4,0.961365,229.189501,0.910
2024-09-17,2024-09-28,235.4,0.963129,226.720641,0.915


# Prepare Export
Prepare a subset of the FDC for export

In [29]:
fdc_out=fdc.loc[:,['VicFalls_full','VicFalls_overlap','VicFalls_factor_smooth','Ngonye_gauged','Ngonye_scaled','Ngonye_synthetic','Conversion']]
fdc_out

,VicFalls_full,VicFalls_overlap,VicFalls_factor_smooth,Ngonye_gauged,Ngonye_scaled,Ngonye_synthetic,Conversion
Exceedance,,,,,,,
0.000,9435.6000,5650.5000,1.669870,5935.9208,9912.215609,9912.215609,1.050512
0.001,8378.3000,5532.4400,1.514395,5751.4615,8709.985085,8709.985085,1.039589
0.002,7352.5016,5315.4790,1.383225,5545.1060,7670.127328,7668.954087,1.043200
0.003,6630.5172,5162.1953,1.284437,5340.5073,6859.547819,6835.562714,1.034542
0.004,6171.6000,4936.6384,1.250162,5073.0458,6342.131411,6342.131411,1.027632
...,...,...,...,...,...,...,...
0.996,126.0000,145.9900,0.863073,155.7670,134.438263,143.592939,1.066970
0.997,115.3000,139.1000,0.828900,154.0979,127.731760,138.801881,1.107821
0.998,107.1000,124.3194,0.861491,152.4361,131.322274,136.495189,1.226165


## Export

In [30]:
#ngonye_synth.to_csv(output_data + 'ngonye_synthetic.csv')
#fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs.csv')
ngonye_synth.to_csv(output_data + 'ngonye_synthetic_2024.csv')
fdc_out.to_csv(output_data + 'ngonye_vicfalls_fdcs_2024.csv')

In [31]:
ngonye_synth.loc[ngonye_synth.index=='1925-02-05']

,LaggedDate,VicFalls,Conversion,Flow,Exceedance
Date,,,,,
1925-02-05,1925-02-16,1238.3,0.940079,1164.100263,0.288


In [32]:
fdc.loc[(fdc.index<=0.291) & (fdc.index>=0.288)].reset_index().sort_values(['VicFalls_full','ExceedanceInv'],ascending=[True,True])

,Exceedance,VicFalls_full,Ngonye_gauged,VicFalls_overlap,VicFalls_factor,VicFalls_factor_smooth,Ngonye_scaled,Conversion,ExceedanceInv,Ngonye_synthetic
3,0.291,1229.000,1418.0238,1506.7775,0.815648,0.814728,1155.303842,0.940036,0.709,1155.303842
2,0.290,1233.744,1421.6563,1516.4000,0.813601,0.815173,1158.895630,0.939332,0.710,1159.887565
1,0.289,1238.300,1428.3252,1516.4000,0.816605,0.814251,1163.015113,0.939203,0.711,1164.100263
0,0.288,1238.300,1429.4710,1528.0000,0.810406,0.814357,1164.100263,0.940079,0.712,1164.100263


In [33]:
vicfalls['Year']=vicfalls.index.year
vicfalls['Month']=vicfalls.index.month
vicfalls['Day']=vicfalls.index.day
vicfalls['WaterYear']=vicfalls.apply((lambda x: int(x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)

In [34]:
years=vicfalls['WaterYear'].unique().tolist()

for year in years:
    days=vicfalls.loc[vicfalls.WaterYear==year]
    days.to_csv(output_data + 'years/vicfalls_' + str(year) + '.csv')